In [13]:
import requests
import sys 
sys.path.append('..')
from bs4 import BeautifulSoup
import re
from datetime import datetime
import pandas as pd
import numpy as np
import pprint
import constants.constants

In [32]:
def pracuj_page_job_offers() -> list:
    months = {
    "stycznia": 1,
    "lutego": 2,
    "marca": 3,
    "kwietnia": 4,
    "maja": 5,
    "czerwca": 6,
    "lipca": 7,
    "sierpnia": 8,
    "września": 9,
    "października": 10,
    "listopada": 11,
    "grudnia": 12,
}
    pracuj_list = list()

    url = "https://www.pracuj.pl/praca/python;kw/warszawa;wp?rd=30&et=1%2c17&tc=0%2c3%2c2&ws=0&wm=full-office%2chybrid%2chome-office"
    flag = True
    try:
        page = requests.get(url)
    except requests.exceptions.ConnectionError as err:
        flag = False
    if flag:
        soup = BeautifulSoup(page.content, "html.parser")

            
        for job_element in soup.select('li[class*="results__list-container-item"]'):

            pracuj_dict = dict()

            link_element = job_element.select('div[class*="offer__info"]')
            
            if len(link_element) != 0:
                job_title_element = job_element.find("a", class_="offer-details__title-link")
                if "Fullstack" not in job_title_element.text and "DevOps" not in job_title_element.text and "Golang" not in job_title_element.text and "Test" not in job_title_element.text:
                    company_element = job_element.find("p", class_="offer-company")
                    publication_date_element = job_element.find("span", class_="offer-actions__date")
                    publication_date_text = publication_date_element.text.strip().split(' ')[1:]
                    month_name = constants.months[publication_date_text[1]]
                    # change polish name of month to number ex. sierpnia to 8
                    publication_date_text[1] = str(month_name)
                    publication_date = '-'.join(publication_date_text)
                    publication_date = datetime.strptime(publication_date, "%d-%m-%Y").strftime("%Y-%m-%d")

                    
                    link_url = job_title_element["href"]
                    link_pattern = re.compile(r'https?://([\w.\.\-]+)')
                    website_name = link_pattern.match(link_url)
                    page_job_element = requests.get(link_url)
                    soup_page = BeautifulSoup(page_job_element.content, "html.parser")
                    position = soup_page.select('li[class*="offer-view"]')
                    if 'unior' in str(position): 
                        position_pattern = 'Junior'
                    elif 'rainee' in str(position) or 'staż' in str(position):
                        position_pattern = 'Trainee'
                    else:
                        position_pattern = 'no info'

                    pracuj_dict["publication_date"] = publication_date
                    pracuj_dict["company"] = company_element.text.strip()
                    pracuj_dict["title"] = job_title_element.text.strip()
                    pracuj_dict["position"] = position_pattern
                    pracuj_dict["link_url"] = link_url
                    pracuj_dict["website"] = website_name[0]
                    pracuj_list.append(pracuj_dict)
    return pracuj_list
   



In [31]:
pracuj_list = pracuj_page_job_offers()
df_raw = pd.DataFrame.from_records(pracuj_list)
df = df_raw.copy()
df.drop_duplicates(subset=['company', 'title'], inplace=True, ignore_index=True)

[<div class="offer__info">
<div class="offer-details">
<div class="offer-logo">
<a href="https://pracodawcy.pracuj.pl/company/1074014825/profile">
<picture>
<source media="(min-width: 1090px)" srcset="https://logos.gpcdn.pl/loga-firm/1074014825/2c580000-43a8-f403-5268-08d92433fdc5_280x280.jpg?width=65&amp;height=65 1x, https://logos.gpcdn.pl/loga-firm/1074014825/2c580000-43a8-f403-5268-08d92433fdc5_280x280.jpg?width=130&amp;height=130 2x"/>
<img alt="logo" class="offer-logo__image" loading="lazy" src="https://logos.gpcdn.pl/loga-firm/1074014825/2c580000-43a8-f403-5268-08d92433fdc5_280x280.jpg" srcset="https://logos.gpcdn.pl/loga-firm/1074014825/2c580000-43a8-f403-5268-08d92433fdc5_280x280.jpg?width=50&amp;height=50 1x, https://logos.gpcdn.pl/loga-firm/1074014825/2c580000-43a8-f403-5268-08d92433fdc5_280x280.jpg?width=100&amp;height=100 2x"/>
</picture>
</a>
</div>
<div class="offer-details__text">
<h2 class="offer-details__title">
<a class="offer-details__title-link" href="https://www.p

AttributeError: module 'constants' has no attribute 'months'

In [25]:
df

""


In [ ]:
list(pracuj_list[0].keys())


['publication_date', 'company', 'title', 'position', 'link_url', 'website']